In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

In [2]:
import torch
from cifar_dataset import CIFAR100, CIFAR100DataModule
from torch.utils.data import DataLoader
import lightning as L
import torch.nn.functional as F
import torch.nn as nn
from lightning.pytorch.tuner import Tuner
from utils.lightning_model import LightningClassifier

/Users/yinnnyou/anaconda3/envs/data_aug_3115/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Pytorch

In [3]:
model = LightningClassifier('pytorch_resnet50', 100)

# train model
trainer = L.Trainer(max_epochs=50)
# tuner = Tuner(trainer)
# print('LR before:', model.lr)
# lr_finder = tuner.lr_find(model, attr_name="lr", 
#               train_dataloaders=train_loader, val_dataloaders=valid_loader)

# Plot with
# print('LR after:', model.lr)
# fig = lr_finder.plot(suggest=True)
# fig.show()
trainer.fit(model, CIFAR100DataModule(batch_size=128))

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type             | Params
---------------------------------------------
0 | model   | ResNet           | 23.7 M
1 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
23.7 M    Trainable params
0         Non-trainable params
23.7 M    Total params
94.852    Total estimated model params size (MB)


Epoch 0:  75%|███████▍  | 234/313 [05:35<01:53,  0.70it/s, v_num=9]        

/Users/yinnnyou/anaconda3/envs/data_aug_3115/lib/python3.11/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


### Timm

In [22]:
model = LightningClassifier('timm_mixer_b16_224_in21k', 100)

# train model
trainer = L.Trainer()
# tuner = Tuner(trainer)
# print('LR before:', model.lr)
# lr_finder = tuner.lr_find(model, attr_name="lr", 
#               train_dataloaders=train_loader, val_dataloaders=valid_loader)

# Plot with
# print('LR after:', model.lr)
# fig = lr_finder.plot(suggest=True)
# fig.show()
trainer.fit(model=model, 
            train_dataloaders=train_loader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/yinnnyou/anaconda3/envs/data_aug_3115/lib/python3.11/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name    | Type             | Params
---------------------------------------------
0 | model   | MlpMixer         | 59.9 M
1 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
59.9 M    Trainable params
0         Non-trainable params
59.9 M    Total params
239.522   Total estimated model params size (MB)
/Users/yinnnyou/anaconda3/envs/data_aug_3115/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in 

Epoch 0:   0%|          | 0/157 [00:00<?, ?it/s] 

AssertionError: Input height (32) doesn't match model (224).